## Search & preview some imagery of farmland near Stockton, CA.

## Motivation
In many workflows, previewing Planet imagery provides immediate value without the need for full order activation or processing. Similar to the web app experience of Planet Explorer, you can preview imagery directly from Data API search results, enabling pre-filtering and informed decisions before placing an order. This guide demonstrates how to leverage Planet’s preview tiles and the Data API to quickly visualize search results on an interactive map, facilitating more efficient data review and planning.


By the end of this guide, you will have an interactive map showing Planet imagery tiles and a clearly demarcated search AOI to assist with the imagery review.

### Overview
Here are the steps we will follow in this quickstart guide:
1. Define an Area of Interest (AOI)
2. Save our AOI's coordinates to GeoJSON format
3. Create a few search filters
4. Search for imagery using those filters
5. Render AOI geometry and preview imagery tiles on a map

**note:** if you are familiar with the [Search and Download Quickstart](./search_and_download_quickstart.ipynb) the first several steps will be identical.

### Requirements
- requests
- folium
- Planet API Key available from your [Planet Account User Settings](https://www.planet.com/account/#/user-settings)

In [ ]:
## install dependencies as needed
!pip install requests folium shapely

## Set Up API Key

In [ ]:
import os

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

assert API_KEY and not API_KEY.startswith('PASTE')

## Define an Area of Interest

An **Area of Interest** (or *AOI*) is how we define the geographic "window" out of which we want to get data.

For the Data API, this could be a simple bounding box with four corners, or a more complex shape, as long as the definition is in [GeoJSON](http://geojson.org/) format. 

For this example, let's just use a simple box. To make it easy, I'll use [geojson.io](http://geojson.io/) to quickly draw a shape & generate GeoJSON output for our box:

![geojsonio.png](images/geojsonio.png)

We only need the "geometry" object for our Data API request:

**ATTENTION** You may need to modify this geometry to be in a region where you have Planet data "Tile View" access.

In [ ]:
# Stockton, CA bounding box (created via geojson.io)
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [ 
      [-121.59290313720705, 37.93444993515032],
      [-121.27017974853516, 37.93444993515032],
      [-121.27017974853516, 38.065932950547484],
      [-121.59290313720705, 38.065932950547484],
      [-121.59290313720705, 37.93444993515032]
    ]
  ]
}

## Create Filters

Now let's set up some **filters** to further constrain our Data API search:

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-08-31T00:00:00.000Z",
    "lte": "2016-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

## Searching: Items and Assets

Planet's products are categorized as **items** and **assets**: an item is a single picture taken by a satellite at a certain time. Items have multiple asset types including the image in different formats, along with supporting metadata files.

For this demonstration, let's get a satellite image that is best suited for analytic applications; i.e., a 4-band image with spectral data for Red, Green, Blue and Near-infrared values. To get the image we want, we will specify an item type of `PSScene`, and asset type `ps4b_analytic` (to get a PSScene4Band Analytic asset).

You can learn more about item & asset types in Planet's Data API [here](https://developers.planet.com/docs/apis/data/items-assets/).

Now let's search for all the items that match our filters:

In [ ]:
import json
import pprint
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
pprint.pprint(geojson['features'][0])

Our search returns metadata for all of the images within our AOI that match our date range and cloud coverage filters. It looks like there are multiple images here; let's extract a list of just those image IDs:

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
print(image_ids)

Since we just want a single image, and this is only a demonstration, for our purposes here we can arbitrarily select the first image in that list. Let's do that, and get the `asset` list available for that image:

In [ ]:
# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())


## Rendering Previews On a Map

First let's make a folium map centered around our area of interest. We use shapely to convert our geojson to a shape and then extract the centroid coordinates.

In [ ]:
import folium
from shapely.geometry import shape

# Convert GeoJSON polygon to shapely geometry to calculate bounds/centroid
polygon = shape(geojson_geometry)
bounds = polygon.bounds
centroid = polygon.centroid

# Calculate center and zoom based on bounds or centroid
map_center = [centroid.y, centroid.x]
print(f"Map Center (based on centroid): {map_center}")

# Initialize the Folium map centered on the centroid of the polygon
folium_map = folium.Map(location=map_center, zoom_start=12)

# Optional: add the GeoJSON polygon to the map for visualization
style_function = lambda feature: {
    'fillOpacity': 0,  # Transparent fill
    'color': 'black',  # Black border
    'weight': 3,       # Thickness of 3 pixels
    'dashArray': '5, 5'  # Dashed border
}
folium.GeoJson(geojson_geometry, style_function=style_function, name="Search Area").add_to(folium_map)

folium_map

Next, we will add preview scene tile layers to the map. To do this, we will leverage [Planet's Tile Services API](https://developers.planet.com/docs/basemaps/tile-services/#api-tile-service)

In [ ]:
PLANET_TILE_URL_TEMPLATE = "https://tiles.planet.com/data/v1/{item_type}/{item_id}/{{z}}/{{x}}/{{y}}.png?api_key={API_KEY}"

In [ ]:
# Add planet tiles from search results
for result in geojson['features']:
    # item_id is the geojson feature id
    item_id = result["id"]
    # item_type is in the geojson feature properties
    item_type = result["properties"]["item_type"]

    # Create the Planet tile URL for this specific item
    tile_url = PLANET_TILE_URL_TEMPLATE.format(item_id=item_id, item_type=item_type, API_KEY=API_KEY)

    # Add the planet tile layer to the map
    folium.TileLayer(
        tiles=tile_url,
        attr='Planet Labs PBC',
        name=f"Planet item {item_type}:{item_id}",
        overlay=True,
        control=True
    ).add_to(folium_map)

# Add layer control so users can toggle between layers
folium.LayerControl().add_to(folium_map)

print("Map with styled GeoJSON and Planet tiles created! Open 'planet_tiles_map_styled.html' to view it.")
folium_map

You can save your map for later viewing outside of the notebook.

**WARNING** if you do save the map, this html artifact will contain your API key in the streaming URLs.

In [ ]:
# Save the map to an HTML file
save_path = 'planet_search_preview_demo_map.html'

# only save with user confirmation
confirm = input(f'Are you sure you want to save this map containing your credentials to path: {save_path}? y/N')
if confirm.lower().strip() != 'y':
    print('NOT SAVING...')
else:
    folium_map.save(save_path)
    print(f'SAVED: {save_path}')

Your final map should look something like this

![folium-map.png](images/folium-map.png)

You can use the folium layer selector to toggle individual results on and off

![layer-selector.png](images/layer-selector.png)

Congrats! Now you know how to stream Planet Preview Tiles to a map so that you can visualize and review your Data API Search Results.